In [0]:
%pip install --quiet --upgrade scikit-learn==1.4.2 tensorflow==2.16.1 scikeras==0.13.0 joblib



In [0]:
dbutils.library.restartPython()

In [0]:
# ==========================================================
#  pix_fraud_inference_v10.py
#  Versão completa — inclui probabilidades por tipo de fraude
# ==========================================================

import os
import json
import numpy as np
import pandas as pd
import mlflow
from datetime import datetime

# ==========================================================
# 1. Configurações de caminhos e artefatos
# ==========================================================
ARTIFACT_DIR = "/Volumes/transacoes_db/copper/files/pix_fraud/production_artifacts"

BINARY_MODEL_PATH = os.path.join(ARTIFACT_DIR, "fraud_binary_pipeline")
MULTICLASS_MODEL_PATH = os.path.join(ARTIFACT_DIR, "fraud_type_pipeline")
LABEL_MAP_PATH = os.path.join(ARTIFACT_DIR, "fraud_type_label_map.json")

# ==========================================================
# 2. Carregamento dos modelos e metadados
# ==========================================================
print(f"Carregando modelo binário de: {BINARY_MODEL_PATH}...")
model_binary = mlflow.sklearn.load_model(BINARY_MODEL_PATH)
print("Modelo binário carregado com sucesso.")

print(f"Carregando modelo multiclasse de: {MULTICLASS_MODEL_PATH}...")
model_multiclass = mlflow.sklearn.load_model(MULTICLASS_MODEL_PATH)
print("Modelo multiclasse carregado com sucesso.")

print(f"Carregando mapa de labels de: {LABEL_MAP_PATH}...")
with open(LABEL_MAP_PATH, "r") as f:
    label_map_raw = json.load(f)

# Normalizar o mapa de labels para int -> nome e criar lista ordenada de classes
label_map_int = {int(k): v for k, v in label_map_raw.items()}
multiclass_class_names = [label_map_int[k] for k in sorted(label_map_int.keys())]
print(f"Mapa de labels carregado. Classes ordenadas: {multiclass_class_names}")

# ==========================================================
# 3. Função de inferência
# ==========================================================
def run_inference(input_df: pd.DataFrame, fraud_threshold: float = 0.5):
    """
    Executa a inferência completa:
    1. Modelo binário → fraude ou legítima
    2. Modelo multiclasse → tipo de fraude (se aplicável)
    Retorna dict com probabilidades e mapeamentos por tipo.
    """
    print(f"\n--- Recebida {len(input_df)} transação(s) para análise. ---")
    print("DEBUG - input_df.dtypes:")
    print(input_df.dtypes)
    print("DEBUG - input_df.head():")
    print(input_df.head().to_string())

    # ======================================================
    # Etapa 1: Modelo Binário (Detecção de Fraude)
    # ======================================================
    print("\nExecutando modelo binário...")
    pred_binary_proba_array = model_binary.predict_proba(input_df)
    print(f"DEBUG - Probabilidades retornadas pelo modelo binário: {pred_binary_proba_array}")

    # --- CORREÇÃO ROBUSTA PARA VÁRIOS FORMATOS DE SAÍDA ---
    proba = pred_binary_proba_array

    if isinstance(proba, np.ndarray) and proba.ndim == 2:
        # formato esperado: [[p0, p1]]
        pred_binary_proba_float = float(round(proba[0][1], 4))
    elif isinstance(proba, np.ndarray) and proba.ndim == 1 and len(proba) == 2:
        # formato: [p0, p1]
        pred_binary_proba_float = float(round(proba[1], 4))
    elif isinstance(proba, np.ndarray) and proba.ndim == 1 and len(proba) == 1:
        # formato: [p1] - probabilidade direta
        pred_binary_proba_float = float(round(proba[0], 4))
    else:
        # Pode ser float ou outro tipo; forçar float
        pred_binary_proba_float = float(round(float(proba), 4))

    print(f"DEBUG - Probabilidade normalizada (fraude=1): {pred_binary_proba_float}")

    pred_binary_label = int(pred_binary_proba_float >= fraud_threshold)
    print(f"Probabilidade de fraude (classe=1): {pred_binary_proba_float}")
    print(f"Predição binária final: {'FRAUDE' if pred_binary_label == 1 else 'LEGÍTIMA'}")

    # ======================================================
    # Etapa 2: Modelo Multiclasse (Classificação do Tipo de Fraude)
    # ======================================================
    pred_multi_label = None
    pred_multi_proba = None
    pred_multi_proba_dict = None

    if pred_binary_label == 1:
        print("\nExecutando modelo multiclasse (tipo de fraude)...")
        pred_multi_proba_array = model_multiclass.predict_proba(input_df)
        print(f"DEBUG - Probabilidades retornadas pelo modelo multiclasse: {pred_multi_proba_array}")

        # Normalizar para vetor de probabilidades por classes
        if isinstance(pred_multi_proba_array, np.ndarray) and pred_multi_proba_array.ndim == 2:
            prob_array = pred_multi_proba_array[0]
        elif isinstance(pred_multi_proba_array, np.ndarray) and pred_multi_proba_array.ndim == 1:
            prob_array = pred_multi_proba_array
        else:
            prob_array = np.array(pred_multi_proba_array).ravel()

        # Caso o número de probabilidades não bata com o número de classes,
        # cortamos/expandimos com zeros (defensivo).
        n_classes = len(multiclass_class_names)
        if prob_array.size != n_classes:
            print(f"Aviso: número de probabilidades ({prob_array.size}) != número de classes ({n_classes}). Ajustando de forma defensiva.")
            # Ajustar: se houver mais probs, truncar; se houver menos, preencher com zeros
            if prob_array.size > n_classes:
                prob_array = prob_array[:n_classes]
            else:
                prob_array = np.concatenate([prob_array, np.zeros(n_classes - prob_array.size)])

        # Mapear nome_da_classe -> probabilidade
        pred_multi_proba_dict = {
            multiclass_class_names[i]: float(round(float(prob_array[i]), 4))
            for i in range(len(multiclass_class_names))
        }

        # Escolher label e confiança
        argmax_idx = int(np.argmax(prob_array))
        pred_multi_label = multiclass_class_names[argmax_idx]
        pred_multi_proba = float(round(float(prob_array[argmax_idx]), 4))

        print(f"Tipo de fraude previsto: {pred_multi_label} (confiança={pred_multi_proba})")
        print(f"Probabilidades por tipo: {pred_multi_proba_dict}")
    else:
        print("Transação legítima — modelo multiclasse não executado.")

    # ======================================================
    # Retornar resultado consolidado
    # ======================================================
    result = {
        "is_fraud": bool(pred_binary_label),
        "fraud_probability": pred_binary_proba_float,
        "fraud_type": pred_multi_label,
        "fraud_type_confidence": pred_multi_proba,
        "fraud_type_probabilities": pred_multi_proba_dict,
        "timestamp_inferencia": datetime.now().isoformat()
    }

    return result

# ==========================================================
# 4. Exemplo de teste
# ==========================================================
if __name__ == "__main__":
    # Simular entrada de uma transação (formato: 1 linha)
    
    data = {
        "id_transacao": ["t2-teste-real-fraude"],
        "pagador_conta_aberta_em": ["2023-05-10T10:00:00"],
        "pagador_segundos_desde_ultima_tx": [300],
        "pagador_data_nascimento": ["1999-07-15T00:00:00"],
        "valor_transacao": [1200000.0],
        "tipo_iniciacao_pix_id": [2],
        "recebedor_txs_ultima_1h": [5],
        "recebedor_idade_conta_dias": [10],
        "pagador_tipo_conta_id": [2],
        "pagador_valor_ultimas_24h": [19500.0],
        "pagador_interacoes_com_recebedor": [1],
        "finalidade_pix_id": [3],
        "recebedor_natureza_id": [1],
        "recebedor_valor_ultima_1h": [19500.0],
        "recebedor_saldo": [19500.0],
        "recebedor_tipo_conta_id": [2],
        "pagador_idade_conta_dias": [90],
        "primeira_interacao": [1],
        "valor_vs_saldo_pagador": [0.9],
        "recebedor_data_nascimento": ["2001-01-01T00:00:00"],
        "pagador_saldo": [20000.0],
        "pagador_txs_ultimas_24h": [1],
        "recebedor_num_pagadores_unicos_24h": [1],
        "recebedor_conta_aberta_em": ["2025-11-01T00:00:00"],
        "pagador_natureza_id": [1],
        "data_transacao": ["2025-11-08T23:30:00"],
        "valor_vs_media_pagador_30d": [25.8],
    }

    df_input = pd.DataFrame(data)
    resultado = run_inference(df_input)

    print("\n--- Resultado Final ---")
    print(json.dumps(resultado, indent=4, ensure_ascii=False))  


In [0]:
import os
import json
import numpy as np
import pandas as pd
import mlflow
from datetime import datetime
import logging
import azure.functions as func
from azure.storage.blob import BlobServiceClient

# ==========================================================
# 1. Configurações de Artefatos (Lendo do Blob)
# ==========================================================

# !! IMPORTANTE !!
# Certifique-se de que estas variáveis de ambiente estão definidas
# nas "Configurações" do seu Aplicativo de Funções no Portal do Azure.
CONNECTION_STRING = os.environ["AZURE_STORAGE_CONNECTION_STRING"]
CONTAINER_NAME = "modelos-fraude" # O nome do contêiner que você criou

# O Azure Functions fornece o diretório /tmp como um disco temporário
LOCAL_ARTIFACT_DIR = "/tmp/pix_fraud_artifacts"
if not os.path.exists(LOCAL_ARTIFACT_DIR):
    os.makedirs(LOCAL_ARTIFACT_DIR, exist_ok=True)

# Caminhos locais para onde os modelos serão baixados
BINARY_MODEL_PATH = os.path.join(LOCAL_ARTIFACT_DIR, "fraud_binary_pipeline")
MULTICLASS_MODEL_PATH = os.path.join(LOCAL_ARTIFACT_DIR, "fraud_type_pipeline")
LABEL_MAP_PATH = os.path.join(LOCAL_ARTIFACT_DIR, "fraud_type_label_map.json")

# ==========================================================
# 2. Funções Auxiliares para Download do Blob
# ==========================================================

def download_blob_file(blob_service_client, container_name, remote_file, local_file):
    """ Baixa um único arquivo do blob. """
    try:
        blob_client = blob_service_client.get_blob_client(container=container_name, blob=remote_file)
        # Cria o diretório local se não existir
        os.makedirs(os.path.dirname(local_file), exist_ok=True)
        with open(local_file, "wb") as download_file:
            download_file.write(blob_client.download_blob().readall())
        logging.info(f"Blob {remote_file} baixado para {local_file}")
    except Exception as e:
        logging.error(f"Falha ao baixar arquivo {remote_file}: {e}")
        raise

def download_blob_directory(blob_service_client, container_name, remote_dir, local_dir):
    """ Baixa um 'diretório' (prefixo) do blob. """
    try:
        container_client = blob_service_client.get_container_client(container_name)
        blob_list = container_client.list_blobs(name_starts_with=remote_dir)
        
        for blob in blob_list:
            # Define o caminho local completo
            relative_path = os.path.relpath(blob.name, remote_dir)
            local_file_path = os.path.join(local_dir, relative_path)
            
            # Cria subdiretórios se não existirem
            local_file_dir = os.path.dirname(local_file_path)
            if not os.path.exists(local_file_dir):
                os.makedirs(local_file_dir, exist_ok=True)
                
            # Baixa o arquivo
            blob_client = container_client.get_blob_client(blob.name)
            with open(local_file_path, "wb") as download_file:
                download_file.write(blob_client.download_blob().readall())
        
        logging.info(f"Diretório blob {remote_dir} baixado para {local_dir}")
    except Exception as e:
        logging.error(f"Falha ao baixar diretório {remote_dir}: {e}")
        raise

# ==========================================================
# 3. Carregamento Global dos Modelos (Executado no Cold Start)
# ==========================================================
logging.info("Iniciando o carregamento global dos modelos (cold start)...")
try:
    # Conectar ao Blob Storage
    blob_service_client = BlobServiceClient.from_connection_string(CONNECTION_STRING)
    
    # --- Baixar Artefatos do Blob ---
    logging.info(f"Baixando modelo binário de {CONTAINER_NAME}/fraud_binary_pipeline...")
    download_blob_directory(blob_service_client, CONTAINER_NAME, "fraud_binary_pipeline", BINARY_MODEL_PATH)
    
    logging.info(f"Baixando modelo multiclasse de {CONTAINER_NAME}/fraud_type_pipeline...")
    download_blob_directory(blob_service_client, CONTAINER_NAME, "fraud_type_pipeline", MULTICLASS_MODEL_PATH)
    
    logging.info(f"Baixando mapa de labels de {CONTAINER_NAME}/fraud_type_label_map.json...")
    download_blob_file(blob_service_client, CONTAINER_NAME, "fraud_type_label_map.json", LABEL_MAP_PATH)
    
    logging.info("Artefatos baixados. Carregando modelos do disco local (/tmp)...")

    # --- Carregar Modelos na Memória ---
    model_binary = mlflow.sklearn.load_model(BINARY_MODEL_PATH)
    model_multiclass = mlflow.sklearn.load_model(MULTICLASS_MODEL_PATH)
    
    with open(LABEL_MAP_PATH, "r") as f:
        label_map_raw = json.load(f)
    
    label_map_int = {int(k): v for k, v in label_map_raw.items()}
    multiclass_class_names = [label_map_int[k] for k in sorted(label_map_int.keys())]
    
    logging.info(f"Modelos carregados com sucesso. Classes: {multiclass_class_names}")

except Exception as e:
    # Se falhar aqui, a função não funcionará.
    logging.critical(f"FALHA CRÍTICA NO COLD START: Não foi possível carregar os modelos. Erro: {e}")
    model_binary = None 
    model_multiclass = None
    multiclass_class_names = []


# ==========================================================
# 4. Função de Inferência (Seu código, com logging)
# ==========================================================
def run_inference(input_df: pd.DataFrame, fraud_threshold: float = 0.5):
    """
    Executa a inferência completa.
    """
    logging.info(f"\n--- Recebida {len(input_df)} transação(s) para análise. ---")
    logging.info("DEBUG - input_df.dtypes:")
    logging.info(input_df.dtypes)

    # ======================================================
    # Etapa 1: Modelo Binário (Detecção de Fraude)
    # ======================================================
    logging.info("\nExecutando modelo binário...")
    pred_binary_proba_array = model_binary.predict_proba(input_df)
    logging.info(f"DEBUG - Probabilidades retornadas pelo modelo binário: {pred_binary_proba_array}")

    # --- CORREÇÃO ROBUSTA PARA VÁRIOS FORMATOS DE SAÍDA ---
    proba = pred_binary_proba_array
    if isinstance(proba, np.ndarray) and proba.ndim == 2:
        pred_binary_proba_float = float(round(proba[0][1], 4))
    elif isinstance(proba, np.ndarray) and proba.ndim == 1 and len(proba) == 2:
        pred_binary_proba_float = float(round(proba[1], 4))
    elif isinstance(proba, np.ndarray) and proba.ndim == 1 and len(proba) == 1:
        pred_binary_proba_float = float(round(proba[0], 4))
    else:
        pred_binary_proba_float = float(round(float(proba), 4))

    logging.info(f"DEBUG - Probabilidade normalizada (fraude=1): {pred_binary_proba_float}")

    pred_binary_label = int(pred_binary_proba_float >= fraud_threshold)
    logging.info(f"Probabilidade de fraude (classe=1): {pred_binary_proba_float}")

    # ======================================================
    # Etapa 2: Modelo Multiclasse (Classificação do Tipo de Fraude)
    # ======================================================
    pred_multi_label = None
    pred_multi_proba = None
    pred_multi_proba_dict = None

    if pred_binary_label == 1:
        logging.info("\nExecutando modelo multiclasse (tipo de fraude)...")
        pred_multi_proba_array = model_multiclass.predict_proba(input_df)
        logging.info(f"DEBUG - Probabilidades retornadas pelo modelo multiclasse: {pred_multi_proba_array}")

        # Normalizar para vetor de probabilidades por classes
        if isinstance(pred_multi_proba_array, np.ndarray) and pred_multi_proba_array.ndim == 2:
            prob_array = pred_multi_proba_array[0]
        elif isinstance(pred_multi_proba_array, np.ndarray) and pred_multi_proba_array.ndim == 1:
            prob_array = pred_multi_proba_array
        else:
            prob_array = np.array(pred_multi_proba_array).ravel()

        # Defensivo
        n_classes = len(multiclass_class_names)
        if prob_array.size != n_classes:
            logging.warning(f"Aviso: número de probabilidades ({prob_array.size}) != número de classes ({n_classes}). Ajustando.")
            if prob_array.size > n_classes:
                prob_array = prob_array[:n_classes]
            else:
                prob_array = np.concatenate([prob_array, np.zeros(n_classes - prob_array.size)])

        # Mapear nome_da_classe -> probabilidade
        pred_multi_proba_dict = {
            multiclass_class_names[i]: float(round(float(prob_array[i]), 4))
            for i in range(len(multiclass_class_names))
        }

        # Escolher label e confiança
        argmax_idx = int(np.argmax(prob_array))
        pred_multi_label = multiclass_class_names[argmax_idx]
        pred_multi_proba = float(round(float(prob_array[argmax_idx]), 4))
        logging.info(f"Probabilidades por tipo: {pred_multi_proba_dict}")
    else:
        logging.info("Transação legítima — modelo multiclasse não executado.")

    # ======================================================
    # Retornar resultado consolidado
    # ======================================================
    result = {
        "is_fraud": bool(pred_binary_label),
        "fraud_probability": pred_binary_proba_float,
        "fraud_type": pred_multi_label,
        "fraud_type_confidence": pred_multi_proba,
        "fraud_type_probabilities": pred_multi_proba_dict,
        "timestamp_inferencia": datetime.now().isoformat()
    }
    return result


# ==========================================================
# 5. Ponto de Entrada da Azure Function (O gatilho HTTP)
# ==========================================================
def main(req: func.HttpRequest) -> func.HttpResponse:
    logging.info('Requisição HTTP recebida para inferência de fraude.')

    # Verificar se os modelos foram carregados corretamente no cold start
    if model_binary is None or model_multiclass is None:
        logging.error("Modelos não estão carregados. Verifique o log de cold start.")
        return func.HttpResponse(
             "Erro interno: Modelos não puderam ser carregados.",
             status_code=500
        )

    try:
        # Receber o JSON do corpo da requisição
        req_body = req.get_json()
        
        # Lógica para garantir que os dados estejam no formato de lista
        # que o pd.DataFrame espera (para uma única linha)
        data_for_df = {}
        for key, value in req_body.items():
            if not isinstance(value, list):
                data_for_df[key] = [value]
            else:
                data_for_df[key] = value

        df_input = pd.DataFrame(data_for_df)

        # Chamar a função de inferência
        resultado = run_inference(df_input)

        # Retornar o JSON
        return func.HttpResponse(
            json.dumps(resultado, indent=4, ensure_ascii=False),
            status_code=200,
            mimetype="application/json"
        )

    except ValueError as ve:
        logging.error(f"Erro de valor ou JSON mal formatado: {ve}")
        return func.HttpResponse(f"JSON inválido ou dados incorretos: {ve}", status_code=400)
    except Exception as e:
        logging.error(f"Erro inesperado na execução: {e}")
        return func.HttpResponse(f"Erro interno no servidor: {e}", status_code=500)

In [0]:
import shutil
import os
import time

# --- 1. Definição dos Caminhos ---

# Origem (no Volume)
source_folder = '/Volumes/transacoes_db/copper/files/pix_fraud/production_artifacts'

# Destino Final (no Volume)
destination_folder = '/Volumes/transacoes_db/copper/files/'
final_zip_name = 'production_artifacts.zip'
final_zip_path = os.path.join(destination_folder, final_zip_name)

# Caminhos Temporários (no disco local do cluster)
local_temp_copy = '/tmp/artifacts_copy'           # Para onde os arquivos serão copiados
local_temp_zip_base = '/tmp/production_artifacts' # Onde o zip será criado
local_zip_file_path = f"{local_temp_zip_base}.zip" # O nome do zip local

start_time = time.time()

try:
    # --- Limpeza de execuções anteriores ---
    if os.path.exists(local_temp_copy):
        print(f"Limpando pasta temporária antiga: {local_temp_copy}")
        shutil.rmtree(local_temp_copy)
    if os.path.exists(local_zip_file_path):
        print(f"Limpando zip temporário antigo: {local_zip_file_path}")
        os.remove(local_zip_file_path)
    
    # --- Etapa 1: Copiar do Volume para o Disco Local ---
    print(f"Copiando {source_folder} para {local_temp_copy}...")
    shutil.copytree(source_folder, local_temp_copy)
    print("Cópia local concluída.")
    
    # --- Etapa 2: Compactar no Disco Local (Local -> Local) ---
    print(f"Compactando {local_temp_copy} para {local_zip_file_path}...")
    shutil.make_archive(
        base_name=local_temp_zip_base, # Onde salvar em /tmp (sem .zip)
        format='zip',
        root_dir='/tmp',
        base_dir='artifacts_copy'      # O que zipar dentro de /tmp
    )
    print("Compactação local concluída.")
    
    # --- Etapa 3: Mover o .zip pronto do Disco Local para o Volume ---
    print(f"Movendo {local_zip_file_path} para {final_zip_path}...")
    shutil.move(local_zip_file_path, final_zip_path)
    
    duration = time.time() - start_time
    print(f"\nSucesso! Arquivo final movido para:")
    print(final_zip_path)
    print(f"Tempo de execução: {duration:.2f} segundos.")

except Exception as e:
    print(f"\nOcorreu um erro: {e}")

finally:
    # --- Limpeza Final ---
    if os.path.exists(local_temp_copy):
        print(f"Limpando pasta temporária final: {local_temp_copy}...")
        shutil.rmtree(local_temp_copy)
    if os.path.exists(local_zip_file_path):
        # Limpa o zip local caso o 'move' falhe
        print(f"Limpando arquivo zip temporário: {local_zip_file_path}...")
        os.remove(local_zip_file_path)